In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [4]:
select NOMEMEDICAMENTO, count(*) as numero
from (select IDPESSOA, NOMEMEDICAMENTO from Medicamento, UsoMedicamento
         where Medicamento.CODIGO = UsoMedicamento.CODMEDICAMENTO
     ) as pessoamed
group by NOMEMEDICAMENTO
order by numero DESC;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [5]:
create view mediusados as select CODIGO, count(*) as numero
from (select IDPESSOA, CODIGO from Medicamento, UsoMedicamento
         where Medicamento.CODIGO = UsoMedicamento.CODMEDICAMENTO
     ) as pessoamed
group by CODIGO
order by numero DESC;

In [6]:
select codigo from (select top 1* from mediusados);

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [7]:
select codigo from (select top 1* from (select CODIGO, count(*) as numero
from (select IDPESSOA, CODIGO from Medicamento, UsoMedicamento
         where Medicamento.CODIGO = UsoMedicamento.CODMEDICAMENTO
     ) as pessoamed
group by CODIGO
order by numero DESC));

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [8]:
select nomemedicamento from (select top 1* from (select nomemedicamento, count(*) as numero
from (select IDPESSOA, nomemedicamento from Medicamento, UsoMedicamento
         where Medicamento.CODIGO = UsoMedicamento.CODMEDICAMENTO
     ) as pessoamed
group by nomemedicamento
order by numero DESC));

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [9]:
select avg(numero) as media from (select IDPESSOA, count(*) as numero from UsoMedicamento group by IDPESSOA);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [10]:
select IDPESSOA, NUMERO from (select IDPESSOA, count(*) as numero from UsoMedicamento group by IDPESSOA) as nmeds
 where nmeds.numero > (select avg(numero) from (select IDPESSOA, count(*) as numero from UsoMedicamento group by IDPESSOA))
 order by numero;

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [11]:
select IDADE, AVG(number) from 
    (select ID, IDADE, count(*) as number from 
         (select ID, IDADE, CODMEDICAMENTO as CODIGO from Pessoa, UsoMedicamento where Pessoa.ID = UsoMedicamento.IDPESSOA)
     group by ID)
group by IDADE
Order by IDADE DESC;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [12]:
create view tuplasmed as select uso1.CODMEDICAMENTO as medicamentoA, uso2.CODMEDICAMENTO as medicamentoB
from UsoMedicamento as uso1, UsoMedicamento as uso2
where uso1.IDPESSOA = uso2.IDPESSOA AND uso1.CODMEDICAMENTO < uso2.CODMEDICAMENTO
order by uso1.CODMEDICAMENTO, uso2.CODMEDICAMENTO;

In [20]:
select * from tuplasmed order by MEDICAMENTOB,MEDICAMENTOA;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [14]:
select MEDICAMENTOA, MEDICAMENTOB, count(*) as numero from tuplasmed
group by MEDICAMENTOA, MEDICAMENTOB
order by numero DESC;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [15]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [16]:
update Matriz
set IdadeAte60=TRUE
WHERE Idade <= 60;
SELECT * FROM Matriz;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [17]:
update Matriz
set MaisUmAnoMedicamento=TRUE
WHERE 365 < (select max(DIASUSO) from UsoMedicamento where UsoMedicamento.IDPESSOA = Matriz.ID );
SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [18]:
update matriz
set MedicamentosAcimaMedia = true
where Matriz.ID in (select IDPESSOA from (select IDPESSOA, count(*) as numero from UsoMedicamento group by IDPESSOA) as nmeds
 where nmeds.numero > (select avg(numero) from (select IDPESSOA, count(*) as numero from UsoMedicamento group by IDPESSOA)))
;
SELECT * FROM Matriz;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Analisar se pessoas mais velhas tem uma tendência maior de tomar remédios por mais de um ano.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [19]:
select idade, MaisUmAnoMedicamento, count(*) as numero from Matriz group by idade, MaisUmAnoMedicamento 
Order by Idade DESC, MAISUMANOMEDICAMENTO DESC;